# Do not run without a VPN, you will get blocked by some of these websites.

In [1]:
import re
import requests
from bs4 import BeautifulSoup
import plotly.express as px
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5055.130 Safari/537.36',
    }
r = requests.get('https://www.gpucheck.com/gpu-benchmark-graphics-card-comparison-chart',headers=headers).text
soup=BeautifulSoup(r,'lxml')

In [3]:
A=soup.findAll('td')
Info=re.findall(r'">(.*?)<',str(A))
Prices=re.findall(r'>\$(.*?)<',str(A))

In [4]:
megalist=[]
for i in range(len(Prices)):
    count = 0
    templist = []
    while count < 5:
        templist.append(Info.pop(0))
        if count == 0:
            templist.append(Prices.pop(0))
        count=count+1
    megalist.append(templist)

In [5]:
df = pd.DataFrame(megalist,columns=('GPU','NEWUSPrice','GPUCheck_Index','AVG1080pFPS','AVG1440pFPS','AVG4kFPS'))
df

,GPU,NEWUSPrice,GPUCheck_Index,AVG1080pFPS,AVG1440pFPS,AVG4kFPS
0,RTX 3090,"1,499.0",287,246.5,187.7,117.9
1,RX 6900 XT,999.0,285,250.1,186.9,105.9
2,RTX 3080 Ti,799.0,283,241.8,182,112.6
3,RX 6800 XT,649.0,276,236.1,176.4,100.1
4,RTX 3080,699.0,275,232.4,173,105
...,...,...,...,...,...,...
104,GTX 760,320.0,40,36.9,27.3,14.9
105,GTX 1050,129.0,39,33.3,24.1,14.1
106,R9 270,275.0,38,32.8,21.9,13.7
107,GTX 950,89.0,36,30,21.4,13.2


### We do not need the GPU Index column. We can create a better ranking column by sorting by a performance column...
##### First, we need to change the majority of the columns to numerical.

In [6]:
df.describe(include='all')

,GPU,NEWUSPrice,GPUCheck_Index,AVG1080pFPS,AVG1440pFPS,AVG4kFPS
count,109,109,109,109,109,109
unique,108,86,77,100,94,98
top,GTX 780M,399.0,45,39,53.5,22.4
freq,2,4,4,3,4,3


In [7]:
GPU_=df.copy()
GPU_.drop('GPUCheck_Index',axis=1,inplace=True)
templist=[]

for entry in list(GPU_['NEWUSPrice']):
    a=str(entry).replace(',','')
    a=float(a)
    templist.append(a)
GPU_['NEWUSPrice']=templist
for entry in ['AVG1080pFPS','AVG1440pFPS','AVG4kFPS']:
    templist=[]
    for i in GPU_[entry]:
        a = float(i)
        templist.append(a)
    GPU_[entry]=templist
    
GPU_.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
GPU,109,108,GTX 780M,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NEWUSPrice,109.0,NaN,NaN,NaN,774.0,588.153277,89.0,320.0,549.0,1199.0,2999.0
AVG1080pFPS,109.0,NaN,NaN,NaN,86.801835,50.781998,30.0,50.2,71.4,105.8,250.1
AVG1440pFPS,109.0,NaN,NaN,NaN,64.23578,38.537628,21.4,36.3,52.8,76.5,187.7
AVG4kFPS,109.0,NaN,NaN,NaN,38.490826,23.154239,12.9,21.6,31.5,46.7,117.9


In [8]:
GPU_.head(10)

,GPU,NEWUSPrice,AVG1080pFPS,AVG1440pFPS,AVG4kFPS
0,RTX 3090,1499.0,246.5,187.7,117.9
1,RX 6900 XT,999.0,250.1,186.9,105.9
2,RTX 3080 Ti,799.0,241.8,182.0,112.6
3,RX 6800 XT,649.0,236.1,176.4,100.1
4,RTX 3080,699.0,232.4,173.0,105.0
5,RX 6800,579.0,197.4,147.6,79.7
6,RTX 3070 Ti,599.0,187.2,139.3,89.0
7,TITAN RTX,2499.0,166.3,127.3,77.8
8,RTX 3070,499.0,178.8,133.1,80.8
9,RTX 2080 Ti,1187.0,161.4,123.6,75.6


In [9]:
list_of_NewPrice_html = {
'RTX 3090':'https://pcpartpicker.com/products/compare/yPpmP6,vyn8TW,YcYmP6,Dt3gXL/',
'RX 6900 XT':'https://pcpartpicker.com/products/compare/CLWBD3,YfvdnQ,pRgFf7,gKnypg/',
'RTX 3080 Ti':'https://pcpartpicker.com/products/compare/PjGbt6,cW92FT,QTmmP6,QFPQzy/',
'RX 6800 XT':'https://pcpartpicker.com/products/compare/XPjNnQ,Lg848d,CLWBD3,YfvdnQ/',
'RTX 3080':'https://pcpartpicker.com/products/compare/m3BG3C,ggNxFT,MYzhP6,8X7G3C/',
'RX 6800':'https://pcpartpicker.com/products/compare/P36qqs,XPjNnQ,FHwkcf,Lg848d/',
'RTX 3070 Ti':'https://pcpartpicker.com/products/compare/wNpzK8,XTmmP6,M2rRsY,CqPQzy/',
'TITAN RTX':'https://pcpartpicker.com/product/TCRzK8/nvidia-titan-rtx-24-gb-video-card-900-1g150-2500-000-sb',
'RTX 3070':'https://pcpartpicker.com/products/compare/7j6qqs,2C7p99,CX4BD3,s4H8TW/',
'RTX 2080 Ti':'https://pcpartpicker.com/products/compare/3G66Mp,Z7gzK8,kZPKHx,pKrmP6/',
'RX 6700 XT':'https://pcpartpicker.com/products/compare/HWbTwP,7whFf7,8HBG3C,cBhFf7/',
'RTX 3060 Ti':'https://pcpartpicker.com/products/compare/9m3gXL,89BG3C,L9BG3C,9F6p99/',
'RTX 2080 SUPER':'https://pcpartpicker.com/products/compare/qHNgXL,KqkgXL,fpbCmG,nzpmP6/',
'TITAN V':'https://pcpartpicker.com/product/YLsmP6/nvidia-titan-v-12gb-video-card-900-1g500-2500-000',
'RTX 2080':'https://pcpartpicker.com/products/compare/YKbwrH,kJZFf7,jmHRsY,jVQG3C/',
'RX 6600 XT':'https://pcpartpicker.com/products/compare/DgFbt6,tWJgXL,QRwypg,nyH7YJ/',
'RTX 2070 SUPER':'https://pcpartpicker.com/products/compare/3sJmP6,tTCFf7,L3tKHx,ZgbCmG/',
'GTX 1080 Ti':'https://pcpartpicker.com/products/video-card/#sort=price&c=390',
'TITAN Xp':'https://pcpartpicker.com/products/compare/TQQRsY,QZ848d,FGfFf7/',
'Radeon VII':'https://pcpartpicker.com/products/compare/cXDJ7P,s2Jtt6,yM97YJ,X6Jtt6/',
'RTX 3060':'https://pcpartpicker.com/products/compare/pD8bt6,tkTp99,4m8bt6,ftJgXL/',
'RX 5700 XT':'https://pcpartpicker.com/products/compare/z6XYcf,3YTzK8,6kdrxr,WGLwrH/',
'RTX 2070':'https://pcpartpicker.com/products/compare/mYYLrH,8nhKHx,JctQzy,jvRgXL/',
'RX 5700':'https://pcpartpicker.com/products/compare/2xrYcf,jHNgXL,K8MTwP,tt7p99/',
'RTX 2060 SUPER':'https://pcpartpicker.com/products/compare/Wzyqqs,9wPgXL,FYrYcf,pL4BD3/',
'GTX 1080':'https://pcpartpicker.com/products/compare/pqmxFT,vzfmP6,v34NnQ,WynG3C/',
'RX 5600 XT':'https://pcpartpicker.com/products/compare/4QK2FT,GzXYcf,xCYQzy,3d3mP6/',
'RTX 2060':'https://pcpartpicker.com/products/compare/RHXnTW,9F7v6h,2H7p99,Xj92FT/',
'RX Vega 64':'https://pcpartpicker.com/products/compare/Fx4NnQ,7nYWGX,MssmP6,dCndnQ/',
'GTX 1070 Ti':'np.nan',
'RX Vega 56':'np.nan',
'GTX TITAN X':'np.nan',
'RTX 3050 Ti':'np.nan',
'RTX 2080 Mobile':'np.nan',
'GTX 1660 Ti':'https://pcpartpicker.com/products/compare/P6CD4D,FdBTwP,YrZFf7,cY3H99/',
'R9 295X2':'np.nan',
'RTX 3050':'https://pcpartpicker.com/products/compare/gmvdnQ,KgH7YJ,7BJgXL,qJZ9TW/',
'GTX 1070':'np.nan',
'GTX 1660 SUPER':'https://pcpartpicker.com/products/compare/jkfFf7,GzPgXL,y2xbt6,YhBhP6/',
'GTX 1080 Mobile':'np.nan',
'GTX 1660':'https://pcpartpicker.com/products/compare/NHdxFT,CnYLrH,DJCD4D,86QfrH/',
'RTX 2080 Max-Q':'np.nan',
'RTX 2070 Mobile':'np.nan',
'GTX 1660 Ti Mobile':'np.nan',
'GTX 1080 Max-Q':'np.nan',
'GTX 1070 Mobile':'np.nan',
'GTX 980 Ti':'np.nan',
'R9 FURY X':'np.nan',
'RX 590':'np.nan',
'RTX 2060 Mobile':'np.nan',
'RTX 2070 Max-Q':'np.nan',
'R9 Nano':'np.nan',
'GTX 1070 Max-Q':'np.nan',
'GTX 980':'np.nan',
'RX 5500 XT 8GB	':'np.nan',
'RX Vega 56 Mobile':'np.nan',
'GTX 1660 Ti Max-Q':'np.nan',
'R9 FURY':'np.nan',
'RX 5500 XT 4GB':'np.nan',
'RX 580':'np.nan',
'GTX 1650 SUPER':'https://pcpartpicker.com/products/compare/LBrYcf,9L7p99,VX2bt6,Gg9tt6/',
'GTX 1060 6GB':'np.nan',
'GTX TITAN BLACK':'np.nan',
'RX 480':'np.nan',
'GTX 1060 3GB':'np.nan',
'HD 7990':'np.nan',
'R9 390X':'np.nan',
'GTX 780 Ti':'np.nan',
'RX 570':'np.nan',
'GTX TITAN':'np.nan',
'GTX 970':'np.nan',
'R9 290X':'np.nan',
'R9 390':'np.nan',
'GTX 1650':'np.nan',
'GTX 1060 Mobile':'np.nan',
'R9 290':'np.nan',
'RX 470':'np.nan',
'GTX 980M':'np.nan',
'GTX 980 Mobile':'np.nan',
'GTX 980MX':'np.nan',
'GTX 780':'np.nan',
'GTX 1060 Max-Q':'np.nan',
'RX 580 Mobile':'np.nan',
'RX 580X Mobile':'np.nan',
'R9 380X':'np.nan',
'RX 480 Mobile':'np.nan',
'R9 280X':'np.nan',
'RX 570 Mobile':'np.nan',
'GTX 770':'np.nan',
'GTX 970M 6GB':'np.nan',
'GTX 970M':'np.nan',
'GTX 1650 Max-Q':'np.nan',
'GTX 1650 Mobile':'np.nan',
'GTX 1050 Ti':'np.nan',
'R9 285':'np.nan',
'R9 380':'np.nan',
'R9 M290X':'np.nan',
'R9 280':'np.nan',
'RX 470 Mobile':'np.nan',
'GTX 960':'np.nan',
'GTX 1050 Ti Mobile':'np.nan',
'GTX 780M':'np.nan',
'GTX 760':'np.nan',
'GTX 1050':'np.nan',
'R9 270':'np.nan',
'GTX 950':'np.nan',
'RX 560':'np.nan'}

In [10]:
headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246',
} #you may need to find a different user agent if you get IP blocked when running this code.

import time

delays = [11,12,14] # you need very high delays with pcpartpicker. they are good at catching scrapers.
delay = np.random.choice(delays) # this will take much longer than the ebay one. 
                                 # remember to refresh your vpn key/port before and after reruns.
for i in range(len(list(list_of_NewPrice_html))):
    name = list(list_of_NewPrice_html.keys())[i]
    entry = list(list_of_NewPrice_html.values())[i]
    if entry != 'np.nan':
        r = requests.get(entry,headers=headers).text
        time.sleep(delay)
        soup=BeautifulSoup(r,'lxml')
        A = re.findall(r"\>\$(.*?)\<\/a\>",str(soup))
        list_of_prices = []
        for item in A:
            list_of_prices.append(float(item.replace('+','')))
        if list_of_prices != []:
            A = min(list_of_prices)
            if A != 0.00:
                list_of_NewPrice_html.update({name:A})
            else:
                list_of_NewPrice_html.update({name:np.nan})
        else:
            list_of_NewPrice_html.update({name:np.nan})
    else:
        list_of_NewPrice_html.update({name:np.nan})
list_of_NewPrice_html

{'RTX 3090': 1131.84,
 'RX 6900 XT': 699.99,
 'RTX 3080 Ti': 869.99,
 'RX 6800 XT': 639.99,
 'RTX 3080': 749.99,
 'RX 6800': 599.99,
 'RTX 3070 Ti': 619.99,
 'TITAN RTX': 3099.0,
 'RTX 3070': 629.99,
 'RTX 2080 Ti': 899.0,
 'RX 6700 XT': 414.99,
 'RTX 3060 Ti': 445.58,
 'RTX 2080 SUPER': 1199.95,
 'TITAN V': 3249.98,
 'RTX 2080': 999.95,
 'RX 6600 XT': 299.99,
 'RTX 2070 SUPER': 1499.99,
 'GTX 1080 Ti': nan,
 'TITAN Xp': nan,
 'Radeon VII': nan,
 'RTX 3060': 369.99,
 'RX 5700 XT': 449.99,
 'RTX 2070': 899.99,
 'RX 5700': nan,
 'RTX 2060 SUPER': 339.99,
 'GTX 1080': nan,
 'RX 5600 XT': 550.0,
 'RTX 2060': 217.99,
 'RX Vega 64': nan,
 'GTX 1070 Ti': nan,
 'RX Vega 56': nan,
 'GTX TITAN X': nan,
 'RTX 3050 Ti': nan,
 'RTX 2080 Mobile': nan,
 'GTX 1660 Ti': 329.99,
 'R9 295X2': nan,
 'RTX 3050': 291.83,
 'GTX 1070': nan,
 'GTX 1660 SUPER': 207.99,
 'GTX 1080 Mobile': nan,
 'GTX 1660': 197.99,
 'RTX 2080 Max-Q': nan,
 'RTX 2070 Mobile': nan,
 'GTX 1660 Ti Mobile': nan,
 'GTX 1080 Max-Q': na

In [11]:
used_price_htmls={'RTX 3090':'https://www.ebay.com/sch/i.html?_from=R40&_nkw=3090&_sacat=0&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&rt=nc&LH_PrefLoc=1',
'RX 6900 XT':'https://www.ebay.com/sch/i.html?_from=R40&_nkw=6900+xt&_sacat=0&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&rt=nc&LH_PrefLoc=1',
'RTX 3080 Ti':'https://www.ebay.com/sch/i.html?_from=R40&_nkw=3080+ti&_sacat=0&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&rt=nc&LH_PrefLoc=1',
'RX 6800 XT':'https://www.ebay.com/sch/i.html?_from=R40&_nkw=6800+xt&_sacat=0&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&rt=nc&LH_PrefLoc=1',
'RTX 3080':'https://www.ebay.com/sch/i.html?_from=R40&_nkw=3080+-3080Ti&_sacat=0&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&rt=nc&LH_PrefLoc=1',
'RX 6800':'https://www.ebay.com/sch/i.html?_from=R40&_nkw=6800&_sacat=0&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&rt=nc&LH_PrefLoc=1',
'RTX 3070 Ti':'https://www.ebay.com/sch/i.html?_from=R40&_nkw=3070+Ti&_sacat=0&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&rt=nc&LH_PrefLoc=1',
'TITAN RTX':'https://www.ebay.com/sch/i.html?_from=R40&_nkw=TITAN+RTX+-bridge+-box+-2080&_sacat=0&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1&rt=nc&Memory%2520Type=GDDR6&_dcat=27386',
'RTX 3070':'https://www.ebay.com/sch/i.html?_from=R40&_nkw=3070+-3060+-box&_sacat=0&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&rt=nc&LH_PrefLoc=1',
'RTX 2080 Ti':'https://www.ebay.com/sch/i.html?_from=R40&_nkw=2080+Ti&_sacat=0&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1&rt=nc&Memory%2520Type=GDDR6&_dcat=27386',
'RX 6700 XT':'https://www.ebay.com/sch/i.html?_from=R40&_nkw=6700+XT&_sacat=0&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1&rt=nc&Memory%2520Type=GDDR6&_dcat=27386',
'RTX 3060 Ti':'https://www.ebay.com/sch/i.html?_from=R40&_nkw=3060+Ti&_sacat=0&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1&rt=nc&Memory%2520Type=GDDR6&_dcat=27386',
'RTX 2080 SUPER':'https://www.ebay.com/sch/i.html?_from=R40&_nkw=2080+SUPER&_sacat=0&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1&rt=nc&Memory%2520Type=GDDR6&_dcat=27386',
'TITAN V':'https://www.ebay.com/sch/i.html?_from=R40&_nkw=titan+v&_sacat=0&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&rt=nc&LH_PrefLoc=1',
'RTX 2080':'https://www.ebay.com/sch/i.html?_from=R40&_nkw=2080+-block&_sacat=0&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&rt=nc&LH_PrefLoc=1',
'RX 6600 XT':'https://www.ebay.com/sch/i.html?_from=R40&_nkw=6600+XT+-bundle+-box&_sacat=0&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&rt=nc&LH_PrefLoc=1',
'RTX 2070 SUPER':'https://www.ebay.com/sch/i.html?_from=R40&_nkw=2070+SUPER+-bundle+-box&_sacat=0&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&rt=nc&LH_PrefLoc=1',
'GTX 1080 Ti':'https://www.ebay.com/sch/i.html?_from=R40&_nkw=1080+Ti+-bundle+-box&_sacat=0&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&rt=nc&LH_PrefLoc=1',
'TITAN Xp':'https://www.ebay.com/sch/i.html?_from=R40&_nkw=TITAN+Xp+gpu+-bundle+-box&_sacat=0&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&rt=nc&LH_PrefLoc=1',
'Radeon VII':'np.nan',
'RTX 3060':'https://www.ebay.com/sch/i.html?_from=R40&_nkw=3060+gpu+-bundle+-box&_sacat=0&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&rt=nc&LH_PrefLoc=1',
'RX 5700 XT':'https://www.ebay.com/sch/i.html?_from=R40&_nkw=5700+XT+gpu+-bundle+-box+-block+-shroud&_sacat=0&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&rt=nc&LH_PrefLoc=1',
'RTX 2070':'https://www.ebay.com/sch/58058/i.html?_from=R40&_nkw=2070+-bios+-tesla+-fan+-bundle+-box+-block+-shroud&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'RX 5700':'https://www.ebay.com/sch/58058/i.html?_from=R40&_nkw=Radeon+Rx+5700&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'RTX 2060 SUPER':'https://www.ebay.com/sch/58058/i.html?_from=R40&_nkw=2060+SUPER+-bios+-tesla+-fan+-bundle+-box+-block+-shroud&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'GTX 1080':'https://www.ebay.com/sch/58058/i.html?_from=R40&_nkw=1080+-fan+-shroud+-waterblock+-block&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'RX 5600 XT':'https://www.ebay.com/sch/58058/i.html?_from=R40&_nkw=5600+XT+-box+-fan+-shroud+-waterblock+-block&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'RTX 2060':'https://www.ebay.com/sch/58058/i.html?_from=R40&_nkw=2060+-fan+-shroud+-waterblock+-block&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'RX Vega 64':'https://www.ebay.com/sch/58058/i.html?_from=R40&_nkw=gpu+Vega+64+-water+-without&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'GTX 1070 Ti':'https://www.ebay.com/sch/58058/i.html?_from=R40&_nkw=1070+ti+-650&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'RX Vega 56':'https://www.ebay.com/sch/58058/i.html?_from=R40&_nkw=RX+Vega+56+-watercool+-fans+-vbios&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'GTX TITAN X':'https://www.ebay.com/sch/58058/i.html?_from=R40&_nkw=GTX+TITAN+X+maxwell&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'RTX 3050 Ti':'np.nan',
'RTX 2080 Mobile':'np.nan',
'GTX 1660 Ti':'https://www.ebay.com/sch/58058/i.html?_from=R40&_nkw=1660+Ti&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'R9 295X2':'np.nan',
'RTX 3050':'https://www.ebay.com/sch/58058/i.html?_from=R40&_nkw=RTX+3050&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'GTX 1070':'https://www.ebay.com/sch/58058/i.html?_from=R40&_nkw=1070+-box+-cooler&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'GTX 1660 SUPER':'https://www.ebay.com/sch/58058/i.html?_from=R40&_nkw=1660+SUPER&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'GTX 1080 Mobile':'np.nan',
'GTX 1660':'https://www.ebay.com/sch/58058/i.html?_from=R40&_nkw=1660&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'RTX 2080 Max-Q':'np.nan',
'RTX 2070 Mobile':'np.nan',
'GTX 1660 Ti Mobile':'np.nan',
'GTX 1080 Max-Q':'np.nan',
'GTX 1070 Mobile':'np.nan',
'GTX 980 Ti':'https://www.ebay.com/sch/58058/i.html?_from=R40&_nkw=980ti&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'R9 FURY X':'https://www.ebay.com/sch/27386/i.html?_from=R40&_nkw=R9+FURY+X&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'RX 590':'https://www.ebay.com/sch/27386/i.html?_from=R40&_nkw=RX+590+-box+-fan+-fans&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'RTX 2060 Mobile':'np.nan',
'RTX 2070 Max-Q':'np.nan',
'R9 Nano':'np.nan',
'GTX 1070 Max-Q':'np.nan',
'GTX 980':'https://www.ebay.com/sch/27386/i.html?_from=R40&_nkw=gtx+980&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'RX 5500 XT 8GB':'https://www.ebay.com/sch/27386/i.html?_from=R40&_nkw=5500+XT+8GB&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'RX Vega 56 Mobile':'np.nan',
'GTX 1660 Ti Max-Q':'np.nan',
'R9 FURY':'np.nan',
'RX 5500 XT 4GB':'https://www.ebay.com/sch/27386/i.html?_from=R40&_nkw=5500+XT&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'RX 580':'https://www.ebay.com/sch/27386/i.html?_from=R40&_nkw=RX+580+-fan+-box&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'GTX 1650 SUPER':'https://www.ebay.com/sch/i.html?_fsrp=1&rt=nc&_from=R40&LH_TitleDesc=0&LH_PrefLoc=1&LH_ItemCondition=1000%7C1500%7C2500%7C3000&_nkw=1650+super&_sacat=27386&LH_BIN=1&_sop=15',
'GTX 1060 6GB':'https://www.ebay.com/sch/27386/i.html?_from=R40&_nkw=GTX+1060+6gb+-box+-fan+-fans&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'GTX TITAN BLACK':'np.nan',
'RX 480':'https://www.ebay.com/sch/27386/i.html?_from=R40&_nkw=rx+480+-material&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'GTX 1060 3GB':'https://www.ebay.com/sch/27386/i.html?_from=R40&_nkw=GTX+1060+3gb+-box+-fan+-fans&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'HD 7990':'np.nan',
'R9 390X':'https://www.ebay.com/sch/27386/i.html?_from=R40&_nkw=R9+390X+-backplate+-accelerator+-box+-fanless&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'GTX 780 Ti':'np.nan',
'RX 570':'https://www.ebay.com/sch/27386/i.html?_from=R40&_nkw=rx+570+-box+-fanless&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'GTX TITAN':'np.nan',
'GTX 970':'https://www.ebay.com/sch/27386/i.html?_from=R40&_nkw=GTX+970&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'R9 290X':'https://www.ebay.com/sch/27386/i.html?_from=R40&_nkw=R9+290X+-backplate+-accelerator+-box+-fanless&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'R9 390':'https://www.ebay.com/sch/27386/i.html?_from=R40&_nkw=R9+390+-backplate+-accelerator+-box+-fanless&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'GTX 1650':'np.nan',
'GTX 1060 Mobile':'np.nan',
'R9 290':'https://www.ebay.com/sch/27386/i.html?_from=R40&_nkw=R9+290+-backplate+-accelerator+-box+-fanless&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'RX 470':'np.nan',
'GTX 980M':'np.nan',
'GTX 980 Mobile':'np.nan',
'GTX 980MX':'np.nan',
'GTX 780':'np.nan',
'GTX 1060 Max-Q':'np.nan',
'RX 580 Mobile':'np.nan',
'RX 580X Mobile':'np.nan',
'R9 380X':'np.nan',
'RX 480 Mobile':'np.nan',
'R9 280X':'np.nan',
'RX 570 Mobile':'np.nan',
'GTX 770':'np.nan',
'GTX 970M 6GB':'np.nan',
'GTX 970M':'np.nan',
'GTX 1650 Max-Q':'np.nan',
'GTX 1650 Mobile':'np.nan',
'GTX 1050 Ti':'https://www.ebay.com/sch/27386/i.html?_from=R40&_nkw=1050+Ti+-box&LH_BIN=1&LH_ItemCondition=1500%7C2500%7C3000%7C1000&LH_ItemCondition=1500%7C2500%7C3000%7C1000&_sop=15&LH_PrefLoc=1',
'R9 285':'np.nan',
'R9 380':'np.nan',
'R9 M290X':'np.nan',
'R9 280':'np.nan',
'RX 470 Mobile':'np.nan',
'GTX 960':'np.nan',
'GTX 1050 Ti Mobile':'np.nan',
'GTX 780M':'np.nan',
'GTX 760':'np.nan',
'GTX 1050':'np.nan',
'R9 270':'np.nan',
'GTX 950':'np.nan',
'RX 560':'np.nan'}

In [12]:
headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5055.130 Safari/537.36'
}

delays = [2,3,4]
delay = np.random.choice(delays)

for i in range(len(list(used_price_htmls.values()))):
    name = list(used_price_htmls.keys())[i]
    entry = list(used_price_htmls.values())[i]
    if entry != 'np.nan':
        html=requests.get(entry,headers=headers).text
        time.sleep(delay)
        soup=BeautifulSoup(html,'lxml')
        A = float(re.findall(r'Amount","value":(.*?),"currency":"USD"}}',str(soup))[0])
    else:
        A = np.nan
    used_price_htmls.update({name:A})
used_price_htmls

{'RTX 3090': 900.0,
 'RX 6900 XT': 699.99,
 'RTX 3080 Ti': 799.99,
 'RX 6800 XT': 589.0,
 'RTX 3080': 615.0,
 'RX 6800': 519.99,
 'RTX 3070 Ti': 450.0,
 'TITAN RTX': 999.99,
 'RTX 3070': 480.0,
 'RTX 2080 Ti': 459.95,
 'RX 6700 XT': 384.24,
 'RTX 3060 Ti': 409.24,
 'RTX 2080 SUPER': 340.0,
 'TITAN V': 239.95,
 'RTX 2080': 340.0,
 'RX 6600 XT': 250.0,
 'RTX 2070 SUPER': 299.99,
 'GTX 1080 Ti': 250.0,
 'TITAN Xp': 429.99,
 'Radeon VII': nan,
 'RTX 3060': 359.0,
 'RX 5700 XT': 295.0,
 'RTX 2070': 260.0,
 'RX 5700': 75.0,
 'RTX 2060 SUPER': 259.98,
 'GTX 1080': 189.99,
 'RX 5600 XT': 200.0,
 'RTX 2060': 15.0,
 'RX Vega 64': 400.0,
 'GTX 1070 Ti': 50.0,
 'RX Vega 56': 215.0,
 'GTX TITAN X': 219.99,
 'RTX 3050 Ti': nan,
 'RTX 2080 Mobile': nan,
 'GTX 1660 Ti': 169.99,
 'R9 295X2': nan,
 'RTX 3050': 260.0,
 'GTX 1070': 170.0,
 'GTX 1660 SUPER': 175.74,
 'GTX 1080 Mobile': nan,
 'GTX 1660': 160.0,
 'RTX 2080 Max-Q': nan,
 'RTX 2070 Mobile': nan,
 'GTX 1660 Ti Mobile': nan,
 'GTX 1080 Max-Q': n

In [13]:
GPU_['Ebay_Used_Price'] = GPU_['GPU'].map(used_price_htmls)
GPU_['PCPartPicker_New_Price'] = GPU_['GPU'].map(list_of_NewPrice_html)

GPU_.to_csv('GPU_Prices_And_Performance.csv')

In [14]:
GPU = pd.read_csv('GPU_Prices_And_Performance.csv',index_col=0)

In [15]:
GPU

,GPU,NEWUSPrice,AVG1080pFPS,AVG1440pFPS,AVG4kFPS,Ebay_Used_Price,PCPartPicker_New_Price
0,RTX 3090,1499.0,246.5,187.7,117.9,900.00,1131.84
1,RX 6900 XT,999.0,250.1,186.9,105.9,699.99,699.99
2,RTX 3080 Ti,799.0,241.8,182.0,112.6,799.99,869.99
3,RX 6800 XT,649.0,236.1,176.4,100.1,589.00,639.99
4,RTX 3080,699.0,232.4,173.0,105.0,615.00,749.99
...,...,...,...,...,...,...,...
104,GTX 760,320.0,36.9,27.3,14.9,NaN,NaN
105,GTX 1050,129.0,33.3,24.1,14.1,NaN,NaN
106,R9 270,275.0,32.8,21.9,13.7,NaN,NaN
107,GTX 950,89.0,30.0,21.4,13.2,NaN,NaN


### I will be sorting by 1080p performance, but you can change this to sort by any performance column you'd like (If you do not have a VPN, I recommend rewriting this code in a separate file and just using the GPU_Prices_and_Performance csv). 
## The lower the Dollars_per_frame, the better.

In [28]:
pd.set_option('display.max_rows',500)
GPU['Dollars_per_frame_used'] = GPU['Ebay_Used_Price'] / GPU['AVG1080pFPS']
# Dollars per frame, new
GPU['Dollars_per_frame_new'] = GPU['PCPartPicker_New_Price'] / GPU['AVG1080pFPS']

New_GPU = GPU[['GPU','AVG1080pFPS','PCPartPicker_New_Price','Dollars_per_frame_new']]
New_GPU = New_GPU.sort_values('Dollars_per_frame_new',ascending=True)

Used_GPU = GPU[['GPU','AVG1080pFPS','Ebay_Used_Price','Dollars_per_frame_used']]
Used_GPU = Used_GPU.sort_values('Dollars_per_frame_used',ascending=True)

New_GPU = New_GPU.sort_values('Dollars_per_frame_new')
New_GPU

,GPU,AVG1080pFPS,PCPartPicker_New_Price,Dollars_per_frame_new
27,RTX 2060,107.8,217.99,2.022171
15,RX 6600 XT,135.8,299.99,2.209057
38,GTX 1660 SUPER,91.0,207.99,2.285604
40,GTX 1660,85.9,197.99,2.304889
10,RX 6700 XT,163.2,414.99,2.542831
3,RX 6800 XT,236.1,639.99,2.710673
60,GTX 1650 SUPER,73.2,199.99,2.732104
1,RX 6900 XT,250.1,699.99,2.798840
24,RTX 2060 SUPER,114.7,339.99,2.964167
11,RTX 3060 Ti,150.0,445.58,2.970533


In [29]:
Used_GPU = Used_GPU.sort_values('Dollars_per_frame_used')
Used_GPU

,GPU,AVG1080pFPS,Ebay_Used_Price,Dollars_per_frame_used
27,RTX 2060,107.8,15.00,0.139147
29,GTX 1070 Ti,101.1,50.00,0.494560
68,RX 570,59.6,32.00,0.536913
23,RX 5700,112.7,75.00,0.665484
61,GTX 1060 6GB,67.0,75.00,1.119403
66,R9 390X,64.2,80.00,1.246106
70,GTX 970,62.2,79.00,1.270096
63,RX 480,59.8,79.00,1.321070
72,R9 390,59.6,80.00,1.342282
53,GTX 980,73.7,100.00,1.356852


### You may also have to clean up some of the ebay values. The searches are very detailed, but sometimes, aberrant values seep through.

In [30]:
Used_GPU = Used_GPU.reset_index().drop('index',axis=1)
Used_GPU.iloc[0,2] = 229.24
Used_GPU.iloc[1,2] = 209.95
Used_GPU.iloc[2,2] = 89.00
Used_GPU.iloc[3,2] = 250.00
Used_GPU.iloc[15,2] = 1449.95
Used_GPU.iloc[18,2] = 144.95
Used_GPU.iloc[19,2] = 289.97
Used_GPU.iloc[23,2] = 269.91

Used_GPU['Dollars_per_frame_used'] = Used_GPU['Ebay_Used_Price'] / Used_GPU['AVG1080pFPS']
Used_GPU = Used_GPU.sort_values('Dollars_per_frame_used',ascending=True)
Used_GPU

,GPU,AVG1080pFPS,Ebay_Used_Price,Dollars_per_frame_used
4,GTX 1060 6GB,67.0,75.00,1.119403
5,R9 390X,64.2,80.00,1.246106
6,GTX 970,62.2,79.00,1.270096
7,RX 480,59.8,79.00,1.321070
8,R9 390,59.6,80.00,1.342282
9,GTX 980,73.7,100.00,1.356852
10,RX 580,71.4,100.00,1.400560
11,GTX 1060 3GB,63.7,89.99,1.412716
12,R9 290X,61.2,90.00,1.470588
2,RX 570,59.6,89.00,1.493289
